In [45]:
import turicreate as tc
import pandas as pd
import numpy as np
import random

class RecommenderClient:
    def __init__(self,data):
        # data is what is returned by Vishnu's client
        self.data = data;
        
        #the ref data is the raw data table
        ref_data = self.ref_data = pd.DataFrame.from_dict(data)
        self.num_of_choices = len(ref_data);
        
        #construct data frame that contains only the training categories
        model_data = self.model_data = ref_data[['category', 'tags']]
        model_data['index'] = model_data.index
        
        #create turicreate SFrame
        tc_data = self.tc_data = tc.SFrame(model_data)
        
        #create model
        model = self.model = tc.recommender.item_content_recommender.create(tc_data, item_id = 'index', verbose=False)
        
    def cold_start(self,k=10):
        #k is the number of items to suggest. 
        #randomly suggests k items
        # returns list of indices. 
        
        suggestion_id = [random.randrange(self.num_of_choices) for _ in range(k)]
        
        return suggestion_id
        
        
    def suggest(self, k = 10, likes = [], dislikes = []):
        #likes is the list of items that have been liked. 
        #returns pandas dataframe with index and the score
        
        recommendations = self.model.recommend_from_interactions(observed_items = likes,
                                                                 k=k,
                                                                 exclude=None,
                                                                 items=None,
                                                                 new_user_data=None,
                                                                 new_item_data=None,
                                                                 exclude_known=True,
                                                                 diversity=0,
                                                                 random_seed=None,
                                                                 verbose=False)
        
        return recommendations

In [46]:
import pickle
import os
os.chdir('../data-import/')
if 0:
    %run main.py 
else:
    data = pickle.load( open( "london_poi.pickle", "rb" ) )
os.chdir('../recommender/')

In [47]:
rc = RecommenderClient(data)

/Users/Devansh/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [48]:
rc.cold_start()

[9, 8, 83, 22, 76, 72, 12, 25, 73, 44]

In [49]:
rc.suggest(k  = 10 , likes = [1,])

index,score,rank
34,0.9472135901451111,1
4,0.9472135901451111,2
31,0.875,3
71,0.7886751294136047,4
58,0.75,5
29,0.75,6
27,0.75,7
26,0.75,8
7,0.75,9
5,0.6767767071723938,10
